In [1]:
class MoveDictionary:
    def __init__(self):
        all_moves = self.generate_all_moves()
        self.move_index_dict = {move: index for index, move in enumerate(all_moves)}
        self.index_move_dict = {index: move for index, move in enumerate(all_moves)}
        #return move_index_dict


    def get_all_legal_moves(self, fen):
        board = chess.Board(fen)
        legal_moves = list(board.legal_moves)  # Get an iterator of legal moves and convert to a list
        moves = [move.uci() for move in legal_moves]
        return [self.move_index_dict[move] for move in moves]

    def generate_all_squares(self):
        files = 'abcdefgh'
        ranks = '12345678'
        return [f + r for f in files for r in ranks]

    def is_within_board(self, file, rank):
        return 'a' <= file <= 'h' and '1' <= rank <= '8'

    def move_in_direction(self, start_square, file_step, rank_step, steps=8):
        moves = []
        start_file, start_rank = start_square[0], start_square[1]
        for step in range(1, steps + 1):
            new_file = chr(ord(start_file) + file_step * step)
            new_rank = chr(ord(start_rank) + rank_step * step)
            if self.is_within_board(new_file, new_rank):
                moves.append(new_file + new_rank)
            else:
                break
        return moves

    def generate_fairy_moves(self, start_square):
        directions = [
            (1, 0), (-1, 0), (0, 1), (0, -1),  # Rook-like moves
            (1, 1), (1, -1), (-1, 1), (-1, -1),  # Bishop-like moves
            (2, 1), (2, -1), (-2, 1), (-2, -1),  # Knight-like moves
            (1, 2), (1, -2), (-1, 2), (-1, -2)
        ]
        moves = []
        for file_step, rank_step in directions:
            if abs(file_step) == 2 or abs(rank_step) == 2:  # Knight-like moves
                moves.extend(self.move_in_direction(start_square, file_step, rank_step, steps=1))
            else:
                moves.extend(self.move_in_direction(start_square, file_step, rank_step))
        return moves

    def generate_promotion_moves(self, start_square, end_square):
        promotion_pieces = ['b', 'n', 'r', 'q']
        return [start_square + end_square + piece for piece in promotion_pieces]

    def generate_all_moves(self):
        all_squares = self.generate_all_squares()
        all_moves = []

        for start_square in all_squares:
            fairy_moves = self.generate_fairy_moves(start_square)
            for end_square in fairy_moves:
                all_moves.append(start_square + end_square)
                # Add promotion moves for pawns
                if start_square[1] == '7' and end_square[1] == '8' and abs(int(ord(start_square[0]))-int(ord(end_square[0]))) <= 1:  # White pawn promotion
                    all_moves.extend(self.generate_promotion_moves(start_square, end_square))
                if start_square[1] == '2' and end_square[1] == '1' and abs(int(ord(start_square[0]))-int(ord(end_square[0]))) <= 1:  # Black pawn promotion
                    all_moves.extend(self.generate_promotion_moves(start_square, end_square))
        return all_moves

In [2]:
def flip_uci(uci_move_string):
    out = ""
    out+=uci_move_string[0]
    out+=str(9 - int(uci_move_string[1]))
    out+=uci_move_string[2]
    out+=str(9 - int(uci_move_string[3]))
    return out

In [3]:
import json
def fen_to_vector(fen):
    fen_parts = fen.split(" ")
    rows = fen_parts[0].split("/")
    turn = fen_parts[1]
    if fen_parts[1] == "b":
        
        rows = [row.swapcase() for row in rows][::-1]
        
        fen_parts[2] = fen_parts[2].swapcase()
        #fen_parts[1] = 1
    # else:
    #     #fen_parts[1] = 0
    position = [0] #special token
    piece_dict = {" ":1, "p":2, "n":3, "b":4, "r":5, "q":6, "k":7, "P":8, "N":9, "B":10, "R":11, "Q":12, "K":13}
   
    for row in rows:
        for square in row:
            if square.isalpha():
                position.append(piece_dict[square])
            else:
                position.extend([1] * int(square))
    castling_rights = fen_parts[2]
    special_tokens = [0,0,0,0]
    for c in castling_rights:
        if c == "K":
            special_tokens[0] = 1
        elif c == "Q":
            special_tokens[1] = 1
        elif c == "k":
            special_tokens[2] = 1
        elif c == "q":
            special_tokens[3] = 1
    en_passant = fen_parts[3]
    if en_passant == "-":
        special_tokens.extend([0] * 9)
    else:
        special_tokens.append(1)
        file_index = ord(en_passant[0]) - 97
        special_tokens.extend([0] * file_index)
        special_tokens.append(1)
        special_tokens.extend([0] * (7 - file_index))

    json_position = json.dumps(position)
    json_special_tokens = json.dumps(special_tokens)

    return json_position, json_special_tokens, turn

In [4]:
import chess.pgn
import time
import pandas as pd
import re
import io

def get_game_result(game):
    """
    Extracts the game result from the game headers.
    Returns:
    - 1 if White wins
    - -1 if Black wins
    - 0 if Draw
    - None if unknown or other result
    """
    result = game.headers["Result"]
    if result == "1-0":
        return 1  # White wins
    elif result == "0-1":
        return -1  # Black wins
    elif result == "1/2-1/2":
        return 0  # Draw
    return None

def game_matches_criteria(game_pgn, expected_termination, min_avg_elo):
    """
    Check if the game matches the termination and Elo criteria.
    """
    termination = re.search(r'\[Termination "([^"]+)"\]', game_pgn)
    if termination and termination.group(1).lower() != expected_termination.lower():
        return False

    white_elo = re.search(r'\[WhiteElo "(\d+)"\]', game_pgn)
    black_elo = re.search(r'\[BlackElo "(\d+)"\]', game_pgn)

    if white_elo and black_elo:
        average_elo = (int(white_elo.group(1)) + int(black_elo.group(1))) / 2
        return average_elo >= min_avg_elo

    return False

In [27]:
def process_single_game(game, game_count, move_to_index):
    board = chess.Board()
    total_moves = len(game["moves"])
    result = game["result"]  # The game result, e.g., "1-0", "0-1", or "1/2-1/2"
    game_data = []

    # Determine who won the game
    if result == "1-0":  # White wins
        winning_player = "w"
    elif result == "0-1":  # Black wins
        winning_player = "b"
    else:  # Draw, we store data from both perspectives
        winning_player = "draw"

    for move_number, move in enumerate(game["moves"][:-1], start=1):  # Exclude the result move
        current_fen = board.fen()  # Get the FEN before making the move
        try:
            # Convert SAN move to UCI move using python-chess
            uci_move = board.parse_san(move)
            moves_left = total_moves - move_number

            # Get board state and turn from fen_to_vector
            board_state, special_tokens, turn = fen_to_vector(current_fen)

            # Only store data from the winning player's perspective or both if it's a draw
            if winning_player == "draw" or winning_player == turn:
                # Extract legal moves and convert them to UCI format
                legal_moves = [m.uci() for m in board.legal_moves]
                num_legal_moves = len(legal_moves)  # Number of legal moves

                # Flip moves if it's Black's turn and handle indexing for moves
                next_move = move_to_index[flip_uci(uci_move.uci())] if turn == "b" else move_to_index[uci_move.uci()]
                legal_moves_converted = [move_to_index[flip_uci(m)] if turn == "b" else move_to_index[m] for m in legal_moves]

                # Store data for the winning player (or both if it's a draw)
                game_data.append({
                    'game_number': int(game_count),
                    'next_move': int(next_move),  # Store the next move index
                    'legal_moves': legal_moves_converted,  # Store legal moves as indices
                    'num_legal_moves': int(num_legal_moves),  # Store the number of legal moves
                    'board_state': board_state,  # Store the board state as a vector
                    'special_tokens': special_tokens,  # Store special tokens (castling, en passant)
                    'fen': current_fen,  # Store the FEN string
                    'moves_left': int(moves_left),  # Moves remaining in the game
                    'result': 1 if turn == winning_player else 0,  # 1 for winner's perspective, 0 for draw
                    'average_elo': int(game["average_elo"]),  # Average Elo of players
                    'turn': turn  # Store whose turn it is ("w" for White, "b" for Black)
                })

            # Apply the move to the board to update the board state
            board.push(uci_move)

        except Exception as e:
            print(f"Error processing move {move} in game {game_count}: {e}")
            continue

    return game_data


In [41]:
def extract_fen_and_moves_with_stats(json_file_path, max_games=None):
    all_game_data = []
    
    # Initialize the move dictionary
    move_dict_obj = MoveDictionary()
    move_to_index = move_dict_obj.move_index_dict
    
    # Load the JSON file
    with open(json_file_path, 'r') as json_file:
        games = json.load(json_file)

    # Initialize timing
    start_time = time.time()

    # Iterate through each game in the JSON file and process them separately
    for game_count, game in enumerate(games, start=1):
        if max_games is not None and game_count > max_games:
            break

        game_data = process_single_game(game, game_count, move_to_index)
        all_game_data.extend(game_data)

        # Time check after every 1000 games
        if game_count % 1000 == 0:
            elapsed_time = time.time() - start_time
            print(f"Processed {game_count} games. Time elapsed: {elapsed_time:.2f} seconds.")

    # Final time check
    total_time = time.time() - start_time
    print(f"Finished processing {game_count} games. Total time taken: {total_time:.2f} seconds.")

    # Convert the list of FENs, moves, and stats into a DataFrame
    df = pd.DataFrame(all_game_data)
    return df



In [ ]:
# Example usage:
json_file = r"output.json"
max_games = 50000  # Limit the number of games processed
game_fen_moves_df = extract_fen_and_moves_with_stats(json_file, max_games=max_games)

# Display the first few rows of the DataFrame
print(game_fen_moves_df.head())


Processed 1000 games. Time elapsed: 6.19 seconds.
Processed 2000 games. Time elapsed: 11.46 seconds.
Processed 3000 games. Time elapsed: 16.70 seconds.
Processed 4000 games. Time elapsed: 22.06 seconds.
Processed 5000 games. Time elapsed: 28.08 seconds.
Processed 6000 games. Time elapsed: 33.46 seconds.
Processed 7000 games. Time elapsed: 38.87 seconds.
Processed 8000 games. Time elapsed: 44.21 seconds.
Processed 9000 games. Time elapsed: 49.49 seconds.
Processed 10000 games. Time elapsed: 55.66 seconds.
Processed 11000 games. Time elapsed: 61.06 seconds.
Processed 12000 games. Time elapsed: 66.52 seconds.
Processed 13000 games. Time elapsed: 71.80 seconds.
Processed 14000 games. Time elapsed: 77.18 seconds.
Processed 15000 games. Time elapsed: 82.58 seconds.
Processed 16000 games. Time elapsed: 87.84 seconds.
Processed 17000 games. Time elapsed: 94.12 seconds.
Processed 18000 games. Time elapsed: 99.44 seconds.
Processed 19000 games. Time elapsed: 104.83 seconds.
Processed 20000 games

In [34]:
game_fen_moves_df

,game_number,next_move,legal_moves,num_legal_moves,board_state,special_tokens,fen,moves_left,result,average_elo,turn
0,1,485,"[1538, 1539, 232, 233, 1788, 1547, 1284, 1016,...",20,"[0, 5, 3, 4, 6, 7, 4, 3, 5, 2, 2, 2, 2, 1, 2, ...","[1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]",rnbqkbnr/pppppppp/8/8/4P3/8/PPPP1PPP/RNBQKBNR ...,47,1,2507,b
1,1,1016,"[1538, 1539, 734, 735, 736, 232, 233, 1788, 15...",22,"[0, 5, 3, 4, 6, 7, 4, 1, 5, 2, 2, 2, 2, 1, 2, ...","[1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]",rnbqkbnr/pp1ppppp/8/2p5/4P3/5N2/PPPP1PPP/RNBQK...,45,1,2507,b
2,1,749,"[1537, 1538, 1539, 1268, 1269, 991, 730, 734, ...",29,"[0, 5, 3, 4, 6, 7, 4, 1, 5, 2, 2, 1, 2, 1, 2, ...","[1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]",rnbqkbnr/pp1p1ppp/4p3/2p5/4P3/2P2N2/PP1P1PPP/R...,43,1,2507,b
3,1,829,"[1537, 1538, 1539, 1268, 1269, 991, 1001, 730,...",34,"[0, 5, 3, 4, 6, 7, 4, 1, 5, 2, 2, 1, 1, 1, 2, ...","[1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]",rnbqkbnr/pp3ppp/4p3/2pp4/3PP3/2P2N2/PP3PPP/RNB...,41,1,2507,b
4,1,565,"[1537, 1538, 1539, 1268, 1269, 991, 730, 723, ...",34,"[0, 5, 3, 4, 6, 7, 4, 1, 5, 2, 2, 1, 1, 1, 2, ...","[1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]",rnbqkbnr/pp3ppp/4p3/2p1N3/3Pp3/2P5/PP3PPP/RNBQ...,39,1,2507,b
...,...,...,...,...,...,...,...,...,...,...,...
76809,2000,1365,"[1365, 1369, 1351, 1353, 1367, 1362, 1373, 1845]",8,"[0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",8/6k1/6p1/8/5K1P/8/8/8 w - - 11 71,5,0,2128,w
76810,2000,1541,"[1559, 1553, 1570, 1540, 1541]",5,"[0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",8/6k1/6p1/6K1/7P/8/8/8 b - - 12 71,4,0,2128,b
76811,2000,1647,"[1651, 1647, 1656, 1845]",4,"[0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 7, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",8/5k2/6p1/6K1/7P/8/8/8 w - - 13 72,3,0,2128,w
76812,2000,1277,"[1297, 1290, 1307, 1277, 1279, 1284, 1302, 1586]",8,"[0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",8/5k2/6p1/8/6KP/8/8/8 b - - 14 72,2,0,2128,b


# Save the data

In [35]:
import sqlite3

def create_database(db_path):
    # Connect to SQLite database at db_path, will create if it doesn't exist
    with sqlite3.connect(db_path) as conn:
        cursor = conn.cursor()
        
        # SQL statement to create a new table with columns matching your data
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS chess_analysis (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                game_number INTEGER,
                board_state TEXT,  -- Store board state as text (JSON or string representation)
                next_move INTEGER,  -- Store the next move index as an integer
                legal_moves TEXT,  -- Store legal moves as text (comma-separated or JSON)
                num_legal_moves INTEGER,  -- Store the number of legal moves as an integer
                moves_left INTEGER,  -- Store the number of moves left in the game as an integer
                result INTEGER,  -- Store the result (1 for win, 0 for draw) as an integer
                average_elo INTEGER,  -- Store average ELO as an integer
                fen TEXT,  -- Store the FEN of the board
                special_tokens TEXT,  -- Store special tokens (castling, en passant) as text (JSON)
                turn TEXT  -- Store whose turn it is ("w" for White, "b" for Black)
            );
        ''')

        conn.commit()
        print("Database created and table initialized for chess analysis.")



In [36]:
import sqlite3
import json

def batch_insert_data(db_path, df, batch_size):
    # Open the database connection
    with sqlite3.connect(db_path) as conn:
        cursor = conn.cursor()
        
        # Prepare the batch of tuples for insertion
        batch = []
        for index, row in df.iterrows():
            # Convert legal_moves list to JSON or comma-separated string
            legal_moves_str = json.dumps(row['legal_moves']) if isinstance(row['legal_moves'], list) else row['legal_moves']

            # Calculate the number of legal moves
            num_legal_moves = len(row['legal_moves']) if isinstance(row['legal_moves'], list) else 0

            # Append each row to the batch
            batch.append((
                row['game_number'],
                row['board_state'],
                row['next_move'],  # Integer field for next move
                legal_moves_str,  # Store the legal moves as JSON or string
                num_legal_moves,  # Number of legal moves
                row['moves_left'],  # Integer field for moves left
                row['result'],  # Integer field: 1 for win, 0 for draw
                row['average_elo'],  # Integer field for average Elo
                row['fen'],  # FEN of the board
                row['special_tokens'],  # JSON string for special tokens
                row['turn']  # Whose turn it is ("w" or "b")
            ))
            
            # Check if the batch size is reached
            if len(batch) >= batch_size:
                cursor.executemany('''
                    INSERT INTO chess_analysis (
                        game_number, board_state, next_move, 
                        legal_moves, num_legal_moves, moves_left, result, 
                        average_elo, fen, special_tokens, turn
                    ) 
                    VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
                ''', batch)
                conn.commit()
                batch = []  # Clear the batch after commit
        
        # Insert any remaining data in the batch
        if batch:
            cursor.executemany('''
                INSERT INTO chess_analysis (
                    game_number, board_state, next_move, 
                    legal_moves, num_legal_moves, moves_left, result, 
                    average_elo, fen, special_tokens, turn
                ) 
                VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
            ''', batch)
            conn.commit()

    print("Batch insertion completed.")


In [37]:
def recreate_table(db_path):
    with sqlite3.connect(db_path) as conn:
        cursor = conn.cursor()
        # Drop the existing table
        cursor.execute('DROP TABLE IF EXISTS chess_analysis')
        conn.commit()
        print("Dropped existing chess_analysis table.")

In [38]:
# Define the database file path
db_path = 'chess_database_final_lichess_77k.db'

# Create the database and table
recreate_table(db_path)
create_database(db_path)

# Assuming you have your DataFrame `game_df`
batch_size = 10000  # Adjust the batch size as needed

# Insert data into the database
batch_insert_data(db_path, game_fen_moves_df, batch_size)
print("Inserted data into database")


Dropped existing chess_analysis table.
Database created and table initialized for chess analysis.
Batch insertion completed.
Inserted data into database


# Check the data

In [39]:
import pandas as pd
import sqlite3
import json

# Your existing function to fetch data from the database
def fetch_data_from_database_to_df(db_path, limit=10):
    """
    Fetches data from the chess_analysis table in the SQLite database and loads it into a DataFrame.
    
    Parameters:
    - db_path: Path to the SQLite database.
    - limit: The number of rows to fetch (default is 10).
    
    Returns:
    - A Pandas DataFrame containing the rows fetched from the database.
    """
    with sqlite3.connect(db_path) as conn:
        query = f"SELECT * FROM chess_analysis LIMIT {limit};"
        df = pd.read_sql_query(query, conn)
        
        return df

# Example usage
db_path = 'chess_database_final_lichess_77k.db'

# Fetch the data from the database
df_retrieved = fetch_data_from_database_to_df(db_path, limit=1000000)
df_retrieved

,id,game_number,board_state,next_move,legal_moves,num_legal_moves,moves_left,result,average_elo,fen,special_tokens,turn
0,1,1,"[0, 5, 3, 4, 6, 7, 4, 3, 5, 2, 2, 2, 2, 1, 2, ...",485,"[1538, 1539, 232, 233, 1788, 1547, 1284, 1016,...",20,47,1,2507,rnbqkbnr/pppppppp/8/8/4P3/8/PPPP1PPP/RNBQKBNR ...,"[1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]",b
1,2,1,"[0, 5, 3, 4, 6, 7, 4, 1, 5, 2, 2, 2, 2, 1, 2, ...",1016,"[1538, 1539, 734, 735, 736, 232, 233, 1788, 15...",22,45,1,2507,rnbqkbnr/pp1ppppp/8/2p5/4P3/5N2/PPPP1PPP/RNBQK...,"[1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]",b
2,3,1,"[0, 5, 3, 4, 6, 7, 4, 1, 5, 2, 2, 1, 2, 1, 2, ...",749,"[1537, 1538, 1539, 1268, 1269, 991, 730, 734, ...",29,43,1,2507,rnbqkbnr/pp1p1ppp/4p3/2p5/4P3/2P2N2/PP1P1PPP/R...,"[1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]",b
3,4,1,"[0, 5, 3, 4, 6, 7, 4, 1, 5, 2, 2, 1, 1, 1, 2, ...",829,"[1537, 1538, 1539, 1268, 1269, 991, 1001, 730,...",34,41,1,2507,rnbqkbnr/pp3ppp/4p3/2pp4/3PP3/2P2N2/PP3PPP/RNB...,"[1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]",b
4,5,1,"[0, 5, 3, 4, 6, 7, 4, 1, 5, 2, 2, 1, 1, 1, 2, ...",565,"[1537, 1538, 1539, 1268, 1269, 991, 730, 723, ...",34,39,1,2507,rnbqkbnr/pp3ppp/4p3/2p1N3/3Pp3/2P5/PP3PPP/RNBQ...,"[1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]",b
...,...,...,...,...,...,...,...,...,...,...,...,...
76809,76810,2000,"[0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",1365,"[1365, 1369, 1351, 1353, 1367, 1362, 1373, 1845]",8,5,0,2128,8/6k1/6p1/8/5K1P/8/8/8 w - - 11 71,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",w
76810,76811,2000,"[0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",1541,"[1559, 1553, 1570, 1540, 1541]",5,4,0,2128,8/6k1/6p1/6K1/7P/8/8/8 b - - 12 71,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",b
76811,76812,2000,"[0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 7, ...",1647,"[1651, 1647, 1656, 1845]",4,3,0,2128,8/5k2/6p1/6K1/7P/8/8/8 w - - 13 72,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",w
76812,76813,2000,"[0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",1277,"[1297, 1290, 1307, 1277, 1279, 1284, 1302, 1586]",8,2,0,2128,8/5k2/6p1/8/6KP/8/8/8 b - - 14 72,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",b


In [40]:
df_retrieved.columns

Index(['id', 'game_number', 'board_state', 'next_move', 'legal_moves',
       'num_legal_moves', 'moves_left', 'result', 'average_elo', 'fen',
       'special_tokens', 'turn'],
      dtype='object')